In [ ]:
import pandas as pd
import json
import re
from tqdm import tqdm
from datasets import load_dataset
from collections import Counter

### Getting KorQuard1.0 and preprocess

In [ ]:
dataset = load_dataset("squad_kor_v1")

In [ ]:
korquard2_train = pd.DataFrame(dataset['train'])
korquard2_val = pd.DataFrame(dataset['validation'])

In [ ]:
korquard2 = pd.concat([korquard2_train, korquard2_val], ignore_index=True)

In [ ]:
title_list = []
context_list = []
question_list = []
id_list = []
answers_list = []
document_id_list = []
for index, row in korquard2.iterrows():
    title_list.append(str(row['title']))
    context_list.append(str(row['context']))
    question_list.append(str(row['question']))
    id_list.append(row['id'])
    answers_list.append(row['answers'])
    document_id_list.append(0)

In [ ]:
processed_korquard = {
    "title" : title_list,
    "context" : context_list,
    "question" : question_list,
    "id" : id_list,
    "answers" : answers_list,
    "document_id" : document_id_list,
}

In [ ]:
processed_korquard_df = pd.DataFrame(processed_korquard)

In [ ]:
processed_korquard_df.head()

### Getting Raw Data

In [ ]:
from datasets import DatasetDict, load_from_disk
datasets = load_from_disk("./data/train_dataset")

In [ ]:
raw_train = pd.DataFrame(datasets['train'])

In [ ]:
# 제목별 빈도수 확인
num_title = sorted(dict(Counter(raw_train['title'])).items(), key=lambda x: x[1], reverse=True)
num_title

### 원본 데이터에서 적은 문서의 경우에만 추가.

In [ ]:
for_add = []
for t, n in num_title:
    # 1. 해당 제목의 데이터가 43보다 적은데 korquard 데이터에 해당 데이터가 43만큼 충분히 채울수 있는 경우
    if n < 43:
        num_korquard = len(processed_korquard_df[processed_korquard_df['title']==t])
        if num_korquard >= (43 - n):
            df = processed_korquard_df[processed_korquard_df['title']==t]
            df = df.sample(n=(43-n), random_state=42)
            for_add.append(df)
        elif 0 < num_korquard < (43-n):
            df = processed_korquard_df[processed_korquard_df['title']==t]
            for_add.append(df)

In [ ]:
for_add.append(raw_train)
sampled_train = pd.concat(for_add, ignore_index=True)

In [ ]:
# 제목별 빈도수 확인
sampled_num_title = sorted(dict(Counter(sampled_train['title'])).items(), key=lambda x: x[1], reverse=True)
sampled_num_title

In [ ]:
temp=0
for t, n in sampled_num_title:
    temp+=n
temp/len(sampled_num_title)

In [ ]:
# 훈련 데이터에 존재하지 않는데 korquard에는 존재하는 데이터들 추가
for_concat=[]
for t in set(processed_korquard_df['title']):
    if t not in set(raw_train['title']) :
        df = processed_korquard_df[processed_korquard_df['title']==t]
        df = df.sample(n=1, random_state=42)
        for_concat.append(df)

In [ ]:
for_concat.append(sampled_train)
sampled_all_data = pd.concat(for_concat, ignore_index=True)
sampled_all_data

In [ ]:
# 제목별 빈도수 확인
final_num_title = sorted(dict(Counter(sampled_all_data['title'])).items(), key=lambda x: x[1], reverse=True)
final_num_title

### 데이터 Huggingface에 올리기

In [ ]:
from datasets import Dataset
concated_dataset = Dataset.from_pandas(sampled_all_data)

In [ ]:
concated_dataset

In [ ]:
# 경로 꼭 바꿔주세요!!!!
# gaeunseo -> 본인 huggingface 계정 이름으로 바꿔주세요
concated_dataset.push_to_hub("gaeunseo/korquard1_and_rawtrain_sampled", private=True)